# 단어와 문자의 원-핫 인코딩
> 원-핫 인코딩은 토큰을 벡터로 변환하는 가장 일반적이고 기본적인 방법이다. 모든 단어에 고유한 정수 인덱스를 부여하고 이 정수 인덱스 i를 크기가 N(어휘사전의 크기)인 이진 벡터로 변환한다. 이 벡터는 i번째 원소만 1이고 나머지는 0이다.
### 단어 수준의 원-핫 인코딩하기(간단한 예시)

In [2]:
import numpy as np

samples = ['The cat sat on the mat.', 'The dog ate my homework.']
# 초기 데이터 : 각 원소가 샘플이다.
token_index = {} # 데이터에 있는 모든 토큰의 인덱스를 구축한다.
for sample in samples:
    for word in sample.split():
        # split()메서드를 사용하여 샘플을 토큰으로 나눈다.
        if word not in token_index:
            token_index[word] = len(token_index) + 1
            # 단어마다 고유한 인덱스를 할당한다. 인덱스 0은 사용하지 않는다.

max_length = 10 # 샘플을 벡터로 변환한다. 각 샘플에서 max_length까지의 단어만 사용한다.

results = np.zeros(shape=(len(samples),
                         max_length,
                         max(token_index.values())+ 1)) # 결과를 저장할 배열이다.
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = token_index.get(word)
        results[i, j, index] = 1.

### 문자 수준 원-핫 인코딩하기(간단한 예시)

In [3]:
import string

samples = ['The cat sat on the mat.', 'The dog ate my homwork.']
characters = string.printable # 출력 가능한 모든 아스키문자
token_index = dict(zip(characters, range(1, len(characters)+1)))

max_length = 50
results = np.zeros((len(samples), max_length, max(token_index.values())+1))
for i, sample in enumerate(samples):
    for j, character in enumerate(sample):
        index = token_index.get(character)
        results[i, j, index] = 1.

### 케라스를 사용한 단어 수준의 원-핫 인코딩

In [5]:
from keras.preprocessing.text import Tokenizer

samples = ['The cat sat on the mat.', 'The dog ate my homework.']

tokenizer = Tokenizer(num_words=1000) # 가장 빈도가 높은 1000개의 단어만 선택하도록 객체를 만든다.
tokenizer.fit_on_texts(samples) # 단어 인덱스를 구축한다.

sequences = tokenizer.texts_to_sequences(samples) # 문자열을 정수 인텍스의 리스트로 변환한다.

one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary')
# 직접 원-핫 이진 벡터 표현을 얻을 수 있다. 다른 벡터화 방법도 제공한다.(count, freq, tfidf)

word_index = tokenizer.word_index # 계산된 단어 인덱스를 구한다.
print('%s개의 고유한 토큰을 찾았습니다.' % len(word_index))

9개의 고유한 토큰을 찾았습니다.


> 원-핫 인코딩의 변종 중 하나로 **원-핫 해싱**기법이 있다. 이 방식은 어휘 사전에 있는 고유한 토큰의 수가 너무 커서 모두 다루기 어려울 때 사용한다. 각 단어에 명시적으로 인덱스를 할당하고 이 인덱스를 딕셔너리에 저장하는 대신에 단어를 해싱하여 고정된 크기의 벡터로 변환한다. 이 방식의 주요 장점은 명시적인 단어 인덱스가 필요없기 때문에 메모리를 절약하고 온라인 방식으로 데이터를 인코딩할 수 있다.
> #### 단점!!! 해시 충돌이다. 2개의 단어가 같은 해시를 만들면 이를 바라보는 모델은 단어 사이의 차이를 인식하지 못한다. 해싱 공간의 차원이 해싱될 고유 토큰의 전체 개수보다 훨씬 크면 가능성이 감소한다.
### 해싱 기법을 사용한 단어 수준의 원-핫 인코딩

In [6]:
samples = ['The cat sat on the mat.', 'The dog ate my homework.']

dimensionality = 1000 # 단어를 크기가 1000인 벡터로 저장한다. 1000개 또는 그 이상의 단어가 있다면 해싱 충돌이 늘어난다.
max_length = 10

results = np.zeros((len(samples), max_length, dimensionality))
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = abs(hash(word)) % dimensionality # 단어를 해싱하여 0과 1000사이의 랜덤한 정수 인덱스로 반환한다.
        results[i, j, index] = 1.